In [0]:
import requests
import json


In [0]:
url = 'https://fake-json-api.mock.beeceptor.com/companies'

In [0]:
response = requests.get('https://fake-json-api.mock.beeceptor.com/companies')
data = response.json()

df = spark.createDataFrame(data)
df.display()


In [0]:
token = dbutils.secrets.get(scpoe = "" , key = "")

headers = {
    "Authorization": f"Bearer {token}"
}
                            
                            

In [0]:
import requests
import json

In [0]:
url = ' https://fake-json-api.mock.beeceptor.com/companies'

responses = requests.get(url)
data = []
if responses.status_code == 200:
    data = responses.json()
    df_company = spark.createDataFrame(data)
    df_company.display()
else:
    print("Error")
    

In [0]:
%sql
drop table dev.bronze.company;

In [0]:
df_company.write.format("delta").option("mode", "overwrite").saveAsTable("dev.bronze.company")

In [0]:
df_comp = spark.read.format("delta").table("dev.bronze.company")
df_comp.display()

In [0]:
url_users = 'https://fake-json-api.mock.beeceptor.com/users'

data = []

if responses.status_code == 200:
    responses = requests.get(url_users)
    data = responses.json()
    df_users = spark.createDataFrame(data)
    df_users.display()
else:
    print("error")



In [0]:
df_users.write.format("delta").saveAsTable("dev.bronze.users")

In [0]:
df_user = spark.read.format("delta").table("dev.bronze.users")
df_user.display()

JOIN TWO DF


In [0]:
def join_df(spark, df1, df2, join_col, join_type):
  result = []
  result = df1.alias("u").join(df2.alias("c"), on=join_col, how=join_type)
  return result

df_user_comp = join_df(spark, df_user, df_comp, "name", "left")
df_user_comp.display()
  

DATA VALIDATION

In [0]:
from pyspark.sql.functions import trim, col

def data_valid(df, trim_cols, valid_col= "email"):
    for c in trim_cols:
        df = df.withColumn(c, trim(col(c)))
        df = df.filter(col(valid_col).endswith("@gmail.com"))
    return df

df_v = data_valid(df_user, df_user.columns, "email") 
    


In [0]:
# Select columns by name or Column object
df_v_selected = df_v.select("email") 

df_trim = df_v.select(df_v.columns)

display(df_v_selected)
display(df_trim)

DROP NULL

In [0]:
def drop_data(df1, *drop_col):
  df_drop = df1.drop(*drop_col)
  return df_drop

df_dr = drop_data(df_user_comp, "address", "company", "country")
df_dr.display()


In [0]:
%sql
describe history dev.bronze.users;

In [0]:
df_dr.select("c.name","u.name").display()

df_dr.select(*df_dr.columns).display()

FILLNA

In [0]:
def fill_data(df1, fill_col, fill_val):
  df_fill = df1.fillna({"fill_col" : "Unknown"})
  return df_fill

In [0]:
%sql
update dev.bronze.users
set country = "" where country = 'Saudi Arabia';

In [0]:
df_us = spark.read.table("dev.bronze.users")
df_us.display()

In [0]:
df_sql = spark.sql(''' update dev.bronze.users 
                   set country = "" 
                   where country = "Panama" ''')
df_sql.display()

In [0]:
df_us = fill_data(df_us, "country", "Unknown")


In [0]:
from pyspark.sql.functions import when, col

def fill_data(df1, fill_col, fill_val):
    df_fill = df1.wiithColumn("fill_col", when(col(fill_col).isNull(), fill_val).otherwise(col(fill_col)))  
    return df_fill

In [0]:
df_fll = fill_data(df_us, df_us.country, "Unknown")

df_fll.select(df_us.country).display()

In [0]:
from pyspark.sql.functions import when, col

def fill_data(df1, fill_col, fill_val):
    return df1.withColumn(
        fill_col,
        when(
            col(fill_col).isNull(),
            fill_val
        ).otherwise(col(fill_col))
    )

df_fll = fill_data(df_us, "country", "Unknown")
df_fll.select("country").display()

DROP DUPLICATES

In [0]:
def frop_dup (df1, group_val, order_col):
  